# Tools

Modelos de linguagem sao bons em gerar texto, mas possuem limitacoes claras: nao conseguem acessar informacoes em tempo real, nao fazem calculos com precisao e nao podem interagir com sistemas externos.

Para superar essas limitacoes, conectamos **ferramentas (tools)** aos agentes. Uma tool e simplesmente uma funcao Python que o agente pode decidir chamar quando julgar necessario. O modelo nao executa a funcao diretamente. Ele gera uma requisicao de chamada (tool call), o LangChain executa a funcao, e o resultado e devolvido ao modelo para que ele formule a resposta final.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Definindo uma tool

Para criar uma tool, basta escrever uma funcao Python normal e decorar com `@tool`. O LangChain usa o **nome da funcao** como nome da tool e a **docstring** como descricao, que e o que o modelo le para decidir quando usar a ferramenta.

A tipagem dos parametros tambem e importante: o modelo usa os tipos para saber quais argumentos passar.

In [2]:
from langchain.tools import tool

@tool
def converter_real_para_dolar(valor_reais: float) -> float:
    """Converte um valor em reais para dolares usando a taxa de cambio fixa de 5.80."""
    taxa = 5.80
    return round(valor_reais / taxa, 2)

Podemos testar a tool diretamente, sem precisar de um agente. Isso e util para validar que a funcao esta funcionando corretamente.

In [3]:
converter_real_para_dolar.invoke({"valor_reais": 1000})

172.41

Tambem e possivel personalizar o nome e a descricao da tool diretamente no decorador, sem depender do nome da funcao e da docstring.

In [4]:
@tool("conversao_brl_usd", description="Converte reais para dolares usando taxa fixa de 5.80")
def minha_tool(valor_reais: float) -> float:
    taxa = 5.80
    return round(valor_reais / taxa, 2)

In [5]:
print(minha_tool.name)
print(minha_tool.description)

conversao_brl_usd
Converte reais para dolares usando taxa fixa de 5.80


## Conectando tools ao agente

Para que o agente possa usar uma tool, basta passa-la na lista `tools` do `create_agent`. A partir dai, o agente decide **autonomamente** quando chamar a ferramenta com base na pergunta do usuario.

In [6]:
from langchain.agents import create_agent

agente = create_agent(
    model="gpt-4.1-nano",
    tools=[converter_real_para_dolar],
    system_prompt="Voce e um assistente financeiro. Use suas ferramentas para fazer conversoes de moeda."
)

In [7]:
from langchain.messages import HumanMessage

resposta = agente.invoke(
    {"messages": [HumanMessage(content="Quanto e 2500 reais em dolares?")]}
)

print(resposta["messages"][-1].content)

2500 reais equivalem a aproximadamente 431.03 dólares, considerando a taxa de câmbio fixa de 5,80.


## Inspecionando o fluxo de chamada

Quando o agente usa uma tool, a lista de mensagens registra todo o fluxo:

1. `HumanMessage` - a pergunta do usuario
2. `AIMessage` com `tool_calls` - o modelo decide qual tool chamar e com quais argumentos
3. `ToolMessage` - o resultado da execucao da tool
4. `AIMessage` - a resposta final do modelo, incorporando o resultado da tool

Vamos inspecionar esse fluxo.

In [8]:
from pprint import pprint

pprint(resposta["messages"])

[HumanMessage(content='Quanto e 2500 reais em dolares?', additional_kwargs={}, response_metadata={}, id='b0ee59ad-2dab-4d7b-bc2b-d484079702ce'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 91, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_74b07c99ad', 'id': 'chatcmpl-DCt5kYwRXmLPLBYrhEpn2mbsD2pMv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c914a-2063-7b21-969d-a971c0fd1c94-0', tool_calls=[{'name': 'converter_real_para_dolar', 'args': {'valor_reais': 2500}, 'id': 'call_zJrN4JGGQ5bFkEe9iHOQmCu7', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 91,

In [9]:
resposta["messages"][1].tool_calls

[{'name': 'converter_real_para_dolar',
  'args': {'valor_reais': 2500},
  'id': 'call_zJrN4JGGQ5bFkEe9iHOQmCu7',
  'type': 'tool_call'}]

Observe que o modelo nao executou a funcao Python. Ele gerou uma **requisicao** com o nome da tool e os argumentos. O LangChain interceptou essa requisicao, executou a funcao, e devolveu o resultado ao modelo.

## A limitacao temporal dos modelos

Modelos de linguagem possuem um **corte de conhecimento** (knowledge cutoff). Eles nao sabem o que aconteceu depois da data em que foram treinados. Isso e um problema para perguntas que exigem informacoes atualizadas.

In [10]:
agente_sem_tools = create_agent(model="gpt-4.1-nano")

resposta = agente_sem_tools.invoke(
    {"messages": [HumanMessage(content="Qual e a cotacao do dolar hoje?")]}
)

print(resposta["messages"][-1].content)

Desculpe, não posso fornecer a cotação atual do dólar, pois minha base de dados foi atualizada pela última vez em outubro de 2023 e não tenho acesso a informações em tempo real. Recomendo verificar a cotação atual em sites de notícias financeiras, bancos ou plataformas de câmbio confiáveis.


O modelo reconhece que nao tem acesso a informacoes em tempo real. Para resolver isso, vamos dar a ele a capacidade de buscar informacoes na web.

## Busca na web com Tavily

Tavily e um servico de busca na web otimizado para uso com LLMs. Ele retorna resultados ja formatados para serem consumidos por modelos de linguagem, diferente de um Google que retorna HTML.

Vamos criar uma tool que usa o Tavily para buscar informacoes atualizadas.

In [11]:
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

In [12]:
@tool
def buscar_na_web(query: str) -> Dict[str, Any]:
    """Busca informacoes atualizadas na internet."""
    return tavily_client.search(query)

Podemos testar a tool diretamente para ver o formato do retorno.

In [13]:
resultado = buscar_na_web.invoke("cotacao do dolar hoje")

pprint(resultado)

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': 'cotacao do dolar hoje',
 'request_id': '445c7cde-35f2-448a-be64-916a91048359',
 'response_time': 1.86,
 'results': [{'content': 'A taxa de cmbio do dólar hoje é 5,18 Real '
                         'Brasileiro. ### Taxas de cmbio de outras moedas '
                         'hoje. * Dólar americanoVerifique a taxa de cmbio de '
                         'Dólar hojeDólar hoje. * Dólar canadenseVerifique a '
                         'taxa de cmbio de Dólar canadense hojeDólar canadense '
                         'hoje. * Dólar australianoVerifique a taxa de cmbio '
                         'de Dólar australiano hojeDólar australiano hoje. * '
                         'Dólar fijianoVerifique a taxa de cmbio de Dólar '
                         'fijiano hojeDólar fijiano hoje. * Dólar '
                         'guianêsVerifique a taxa de cmbio de Dólar guianense '
                         'hojeDólar guianense hoje. * Dó

## Agente com busca na web

Agora vamos conectar a tool de busca ao agente. A mesma pergunta que antes gerou uma resposta evasiva agora vai retornar dados atualizados.

In [14]:
agente = create_agent(
    model="gpt-4.1-nano",
    tools=[buscar_na_web]
)

In [15]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual e a cotacao do dolar hoje?")]}
)

print(resposta["messages"][-1].content)

A cotação do dólar hoje está aproximadamente em 5,18 reais, de acordo com a Wise. Para informações mais precisas e atualizadas, recomendo consultar uma fonte financeira confiável ou o banco de sua preferência.


In [16]:
pprint(resposta["messages"])

[HumanMessage(content='Qual e a cotacao do dolar hoje?', additional_kwargs={}, response_metadata={}, id='f5e64d47-f901-4279-a808-af90d456c0b9'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 55, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_b0b94f3709', 'id': 'chatcmpl-DCt68wUXOnCJMxGZwkj4Hw0lrbwwS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c914a-7ba1-7440-9509-91df85953ec9-0', tool_calls=[{'name': 'buscar_na_web', 'args': {'query': 'cotação do dólar hoje'}, 'id': 'call_F5tp1HdE8xuWbYVCB8DHYjg4', 'type': 'tool_call'}, {'name': 'buscar_na_web', 'args': {'query': 'valor do dóla

Agora o agente tem acesso a informacoes em tempo real. Ele decidiu autonomamente buscar na web, interpretar os resultados e formular uma resposta com dados atualizados.

Esse padrao, de dar capacidades extras ao modelo por meio de tools, e o fundamento de como agentes de IA funcionam na pratica. Nas proximas aulas vamos expandir isso com mais ferramentas, memoria e fluxos mais complexos.